#### Feature Engineering  
Dataset: _music_clean_   
Author: Luis Sergio Pastrana Lemus  
Date: 2025-04-23

## __1. Libraries__

In [1]:
from IPython.display import display, HTML
import os
import pandas as pd
from pathlib import Path
import sys

# Define project root dynamically, gets the current directory from whick the notebook belongs and moves one level upper
project_root = Path.cwd().parent

# Add src to sys.path if it is not already
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import function directly (more controlled than import *)
from src import *

## __2. Path to Data file__

In [2]:
# Build route to data file and upload
data_file_path = project_root / "data" / "processed"
df_music = load_dataset_from_csv(data_file_path, "music_clean.csv", sep=',', header='infer', keep_default_na=False)


In [3]:
# Format notebook output
format_notebook()

## 3 __Casting to data types__

### 3.1 Casting to category data type

In [4]:
# Casting to category dtype
df_music['genre'] = df_music['genre'].astype('category')
df_music['city'] = df_music['city'].astype('category')
df_music['day'] = df_music['day'].astype('category')

In [5]:
# Checking dtypes after imputation
display(HTML(f"> Data types after imputation:\n"))
print(df_music.loc[:, ['genre', 'city', 'day']].dtypes)

genre    category
city     category
day      category
dtype: object


### 3.1 Casting to datetime data type

In [6]:
df_music = normalize_datetime(df_music, include=["time"], frmt="%H:%M:%S")

type(df_music['time'].iloc[0])

datetime.time

In [7]:
df_music['hour'] = df_music['time'].apply(lambda x: x.hour if pd.notnull(x) else None)
df_music

,userid,track,artist,genre,city,time,day,hour
0,FFB692EC,kamigata_to_boots,the_mass_missile,rock,shelbyville,20:28:33,wednesday,20
1,55204538,delayed_because_of_accident,andreas_rönnberg,rock,springfield,14:07:09,friday,14
2,20EC38,funiculì_funiculà,mario_lanza,pop,shelbyville,20:58:07,wednesday,20
3,A3DD03C9,dragons_in_the_sunset,fire_ice,folk,shelbyville,08:37:09,monday,8
4,E2DC1FAE,soul_people,space_echo,dance,springfield,08:34:34,monday,8
...,...,...,...,...,...,...,...,...
59986,729CBB09,my_name,mclean,rnb,springfield,13:32:28,wednesday,13
59987,D08D4A55,maybe_one_day_feat_black_spade_,blu_exile,hiphop,shelbyville,10:00:00,monday,10
59988,C5E3A0D5,jalopiina,unknown,industrial,springfield,20:09:26,friday,20
59989,321D0506,freight_train,chas_mcdevitt,rock,springfield,21:43:59,friday,21


In [8]:
df_music['hour'] = df_music['hour'].astype('category')
# Checking dtypes after imputation
display(HTML(f"> Data types after imputation:\n"))
print(df_music.loc[:, ['hour']].dtypes)

hour    category
dtype: object


## 3. Feature Engineering

### 3.1 User activity variation by day and city

#### 3.1.1 User activity variation by city

In [9]:
df_music

,userid,track,artist,genre,city,time,day,hour
0,FFB692EC,kamigata_to_boots,the_mass_missile,rock,shelbyville,20:28:33,wednesday,20
1,55204538,delayed_because_of_accident,andreas_rönnberg,rock,springfield,14:07:09,friday,14
2,20EC38,funiculì_funiculà,mario_lanza,pop,shelbyville,20:58:07,wednesday,20
3,A3DD03C9,dragons_in_the_sunset,fire_ice,folk,shelbyville,08:37:09,monday,8
4,E2DC1FAE,soul_people,space_echo,dance,springfield,08:34:34,monday,8
...,...,...,...,...,...,...,...,...
59986,729CBB09,my_name,mclean,rnb,springfield,13:32:28,wednesday,13
59987,D08D4A55,maybe_one_day_feat_black_spade_,blu_exile,hiphop,shelbyville,10:00:00,monday,10
59988,C5E3A0D5,jalopiina,unknown,industrial,springfield,20:09:26,friday,20
59989,321D0506,freight_train,chas_mcdevitt,rock,springfield,21:43:59,friday,21


In [10]:
df_music_city = pd.pivot_table(df_music, index='city', values=['userid', 'track'], aggfunc={'userid': pd.Series.nunique, 'track': [pd.Series.nunique, 'count']}, observed=False)
df_music_city


track          userid
             count nunique nunique
city                              
shelbyville  18118   14406   12293
springfield  41873   29208   29069

In [11]:
df_music_city.columns = df_music_city.columns = ['total_tracks', 'tracks', 'users']
df_music_city


,total_tracks,tracks,users
city,,,
shelbyville,18118,14406,12293
springfield,41873,29208,29069


In [12]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "music_activity_city.csv"
df_music_city.to_csv(processed_path)


#### 3.1.2 User activity variation by day

In [13]:
df_music_day = pd.pivot_table(df_music, index='day', values=['userid', 'track'], aggfunc={'userid': pd.Series.nunique, 'track': [pd.Series.nunique, 'count']}, observed=False)
df_music_day

track          userid
           count nunique nunique
day                             
friday     21475   17141   16473
monday     20799   16570   15925
wednesday  17717   14370   13490

In [14]:
df_music_day.columns = ['total_tracks', 'tracks', 'users']
df_music_day

,total_tracks,tracks,users
day,,,
friday,21475,17141,16473
monday,20799,16570,15925
wednesday,17717,14370,13490


In [15]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "music_activity_day.csv"

df_music_day.to_csv(processed_path)

#### 3.1.3 User activity variation by city and day

In [16]:
df_music_city_day = pd.pivot_table(df_music, index=['city', 'day'], values=['userid', 'track'], aggfunc={'userid': pd.Series.nunique, 'track': [pd.Series.nunique, 'count']}, observed=False)
df_music_city_day

track          userid
                       count nunique nunique
city        day                             
shelbyville friday      5786    5208    4447
            monday      5462    4896    4186
            wednesday   6870    6055    5177
springfield friday     15689   12908   12030
            monday     15337   12593   11747
            wednesday  10847    9236    8314

In [17]:
df_music_city_day = df_music_city_day.reset_index()
df_music_city_day

city        day  track          userid
                           count nunique nunique
0  shelbyville     friday   5786    5208    4447
1  shelbyville     monday   5462    4896    4186
2  shelbyville  wednesday   6870    6055    5177
3  springfield     friday  15689   12908   12030
4  springfield     monday  15337   12593   11747
5  springfield  wednesday  10847    9236    8314

In [ ]:
df_music_city_day.columns = ['city', 'day', 'total_tracks', 'tracks', 'users']
df_music_city_day

,city,day,total_tracks,tracks,users
0,shelbyville,friday,5786,5208,4447
1,shelbyville,monday,5462,4896,4186
2,shelbyville,wednesday,6870,6055,5177
3,springfield,friday,15689,12908,12030
4,springfield,monday,15337,12593,11747
5,springfield,wednesday,10847,9236,8314


In [20]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "music_activity_city_day.csv"

df_music_city_day.to_csv(processed_path, index=False)

### 3.2 User activity variation by time, day and city

#### 3.2.1 User activity variation by time and city

In [ ]:
df_music_time_city = pd.pivot_table(df_music, index=['city', 'hour'], values=['userid', 'track'], aggfunc={'userid': pd.Series.nunique, 'track': [pd.Series.nunique, 'count']}, observed=False)
df_music_time_city

In [ ]:
df_music_time_city = df_music_time_city.reset_index()
df_music_time_city

In [ ]:
df_music_time_city.columns = ['city', 'hour', 'total_tracks', 'tracks', 'users']
df_music_time_city

In [ ]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "music_activity_time_city.csv"

df_music_time_city.to_csv(processed_path)

#### 3.2.2 User activity variation by time and day

In [ ]:
df_music_time_day = pd.pivot_table(df_music, index=['day', 'hour'], values=['userid', 'track'], aggfunc={'userid': pd.Series.nunique, 'track': [pd.Series.nunique, 'count']}, observed=False)
df_music_time_day

In [ ]:
df_music_time_day = df_music_time_day.reset_index()
df_music_time_day

In [ ]:
df_music_time_day.columns = ['day', 'hour', 'total_tracks', 'tracks', 'users']
df_music_time_day

In [ ]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "music_activity_time_day.csv"

df_music_time_day.to_csv(processed_path)